In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import (
    AIMessage, HumanMessage, SystemMessage, ToolMessage
)
from langchain.tools import tool
from langchain_core.output_parsers.openai_tools import parse_tool_calls

In [2]:
from dotenv import load_dotenv
load_dotenv()
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

In [3]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [4]:
tools = [multiply]
tool_map = {tool.name:tool for tool in tools}

In [5]:
tool_map

{'multiply': StructuredTool(name='multiply', description='Multiply two numbers.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x1209b9ab0>)}

In [6]:
llm_with_tools = llm.bind_tools(tools)

In [7]:
question = "3 multiplied by 2"

In [8]:
messages = [
    SystemMessage("You're a helpful assistant"),
    HumanMessage("3 multiplied by 2")
]

In [9]:
ai_message = llm_with_tools.invoke(messages)

In [10]:
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zo5b2B3Ysxap9ELPN8NeuL7m', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 54, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_13eed4fce1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b002dd23-3286-4d0b-9d21-7292995d7c33-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_zo5b2B3Ysxap9ELPN8NeuL7m', 'type': 'tool_call'}], usage_metadata={'input_tokens': 54, 'output_tokens': 18, 'total_tokens': 72, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [11]:
messages.append(ai_message)

In [12]:
messages

[SystemMessage(content="You're a helpful assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='3 multiplied by 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zo5b2B3Ysxap9ELPN8NeuL7m', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 54, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_13eed4fce1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b002dd23-3286-4d0b-9d21-7292995d7c33-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_zo5b2B3Ysxap9ELPN8NeuL7m', 'type': 'tool_call'}], usage_m

In [13]:
parsed_tool_calls = parse_tool_calls(
    ai_message.additional_kwargs.get("tool_calls")
)

In [14]:
for tool_call in parsed_tool_calls:
    tool_call_id = tool_call['id']
    function_name = tool_call['name']
    arguments = tool_call['args']
    func = tool_map[function_name]
    result = func.invoke(arguments)
    tool_message = ToolMessage(
        content=result,
        name=function_name,
        tool_call_id=tool_call_id,
    )
    messages.append(tool_message)

In [15]:
messages

[SystemMessage(content="You're a helpful assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='3 multiplied by 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zo5b2B3Ysxap9ELPN8NeuL7m', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 54, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_13eed4fce1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b002dd23-3286-4d0b-9d21-7292995d7c33-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 2}, 'id': 'call_zo5b2B3Ysxap9ELPN8NeuL7m', 'type': 'tool_call'}], usage_m

In [16]:
ai_message = llm_with_tools.invoke(messages)

In [17]:
ai_message

AIMessage(content='3 multiplied by 2 is 6.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 79, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a', 'finish_reason': 'stop', 'logprobs': None}, id='run-50e217de-edf9-4a46-afc2-ad287f184d78-0', usage_metadata={'input_tokens': 79, 'output_tokens': 11, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})